In [2]:
import pandas as pd 
import process
import numpy as np 
# Jerome path : r'C:\Users\33640\OneDrive\Documents\GitHub\Portfolio_clustering_project\Data\DataBase.csv'
# Nail path : '/Users/khelifanail/Documents/GitHub/Portfolio_clustering_project/Data/DataBase.csv'
df = pd.read_csv(r'/Users/khelifanail/Documents/GitHub/Portfolio_clustering_project/Data/DataBase.csv')

df.set_index('ticker', inplace=True)

df.columns = pd.to_datetime(df.columns.str[1:], format='%Y%m%d').strftime('%d/%m/%Y')

df_cleaned = df.fillna(0) # Utilisez la méthode fillna(0) pour remplacer les NaN par 0

df_cleaned = df_cleaned.transpose() ## WE WANT COLUMNS TO BE VECTOR OF RETURN FOR A GIVEN TICKER

In [8]:
beta = 0.95
K = 4  # Number of fold for the cross validation
N = 695  # Number of assets
Ik_length = 50  # Number of days in each fold for the cross validation

# Initialize epsilon as a zero array with N elements
epsilon = np.zeros(N)


for k in range(K):
    # Calculate EWA matrix 
    weighted_matrices = [(beta**(Ik_length-t)) * np.outer(df_cleaned.iloc[t + Ik_length*k], df_cleaned.iloc[t + Ik_length*k]) for t in range(Ik_length)]
    summed_weighted_matrices = np.sum(weighted_matrices, axis=0)
    E_matrix = (1 - beta) / (1 - beta**Ik_length) * summed_weighted_matrices
    
    # Calculate eigenvectors for the E matrix
    eigenvalues, eigenvectors = np.linalg.eigh(E_matrix)

    # Calculate epsilon terms for each eigenvector
    for i in range(N):
        ui = eigenvectors[:, i]
        # For each day in the Ik segment, project the data onto the eigenvector and square it
        epsilon_i_sum = np.sum([(np.dot(ui, df_cleaned.iloc[t + Ik_length*k])**2) for t in range(Ik_length)])
        # Accumulate the results in epsilon
        epsilon[i] += epsilon_i_sum.real / Ik_length

# Average epsilon over K segments
epsilon /= K

# Now, we calculate the forecasts using the last set of eigenvectors
Forecasts = pd.DataFrame(index=df_cleaned.columns, columns=df_cleaned.columns, data=np.sum([epsilon[i] * np.outer(eigenvectors[:, i], eigenvectors[:, i]) for i in range(N)], axis=0))

In [9]:
Forecasts

ticker,AA,ABM,ABT,ADI,ADM,ADX,AEE,AEG,AEM,AEP,...,XLI,XLK,XLP,XLU,XLV,XLY,XOM,XRX,YUM,ZTR
ticker,,,,,,,,,,,,,,,,,,,,,
AA,0.001317,-0.000149,-0.000036,-0.000165,0.000050,0.000139,-0.000010,0.000161,-0.000105,0.000041,...,0.000089,-0.000032,-0.000052,0.000078,0.000055,0.000102,0.000027,0.000052,-0.000031,0.000156
ABM,-0.000149,0.000390,0.000082,0.000219,-0.000040,-0.000002,-0.000015,-0.000009,-0.000049,-0.000071,...,0.000003,0.000028,0.000013,0.000016,-0.000002,0.000027,0.000007,0.000271,0.000032,-0.000005
ABT,-0.000036,0.000082,0.000387,0.000026,0.000030,0.000031,0.000014,-0.000003,0.000006,0.000080,...,-0.000012,-0.000034,0.000089,0.000024,0.000055,0.000008,-0.000014,-0.000031,0.000069,-0.000030
ADI,-0.000165,0.000219,0.000026,0.003124,0.000305,0.000127,-0.000030,0.000009,0.000060,-0.000150,...,0.000219,0.000656,-0.000095,-0.000012,0.000243,0.000121,-0.000031,0.001396,0.000415,0.000097
ADM,0.000050,-0.000040,0.000030,0.000305,0.000440,0.000040,0.000030,0.000078,0.000006,0.000006,...,0.000071,0.000070,0.000008,0.000030,0.000073,0.000031,0.000007,-0.000143,0.000244,0.000012
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
XLY,0.000102,0.000027,0.000008,0.000121,0.000031,0.000057,0.000004,0.000035,0.000022,0.000030,...,0.000050,0.000072,-0.000013,0.000040,0.000061,0.000167,-0.000010,-0.000038,0.000057,0.000031
XOM,0.000027,0.000007,-0.000014,-0.000031,0.000007,-0.000039,0.000019,-0.000022,0.000052,0.000011,...,-0.000036,-0.000081,0.000023,0.000011,-0.000030,-0.000010,0.000184,0.000373,-0.000057,-0.000021
XRX,0.000052,0.000271,-0.000031,0.001396,-0.000143,-0.000127,-0.000008,-0.000103,0.000131,-0.000139,...,-0.000173,0.000260,-0.000144,0.000013,-0.000015,-0.000038,0.000373,0.004752,-0.000245,0.000136


In [2]:
from PyFolioC import PyFolio
from PyFolioC import PyFolioC

historical_data = df_cleaned
number_of_repetitions = 10
lookback_window = [0, 50]
evaluation_window = 1
number_of_clusters = 38
clustering_method = 'SPONGE'
sigma = 0.1
eta = 0.1

consolidated_W = PyFolioC(number_of_repetitions=number_of_repetitions, historical_data=df_cleaned, lookback_window=lookback_window, evaluation_window=evaluation_window, number_of_clusters=number_of_clusters, sigma=sigma, eta=eta, clustering_method=clustering_method)

In [3]:
consolidated_W.sliding_window(10)

1.0044931623282323
step 1
1.0166687281847513
step 2
1.0195083623502044
step 3
1.0203762217597587
step 4
1.0256214503165464
step 5
1.0278326613983257
step 6
1.030928249800027
step 7
1.0304429207078931
step 8
1.0314772312816862
step 9
1.0320651616089225
step 10


(              return
 15/03/2000  0.004493
 16/03/2000  0.012121
 17/03/2000  0.002793
 20/03/2000  0.000851
 21/03/2000  0.005140
 22/03/2000  0.002156
 23/03/2000  0.003012
 24/03/2000 -0.000471
 27/03/2000  0.001004
 28/03/2000  0.000570,
 array([0.  , 0.02, 0.02, 0.02, 0.03, 0.03, 0.03, 0.03, 0.03, 0.03]),
 [1,
  1.0044931623282323,
  1.0166687281847513,
  1.0195083623502044,
  1.0203762217597587,
  1.0256214503165464,
  1.0278326613983257,
  1.030928249800027,
  1.0304429207078931,
  1.0314772312816862,
  1.0320651616089225],
 array([ 0.  ,  0.01,  0.  ,  0.  ,  0.01,  0.  ,  0.  , -0.  ,  0.  ,
         0.  ]))

In [ ]:
def calcul_somme(beta):

    T = len(consolidated_W.historical_data.columns)
    
    res = np.zeros((len(df_column[0]), len(df_column[0])))

    res = (1 - beta) / (1 - beta**T) * np.sum([(beta**(T-(t+1))) * np.outer(consolidated_W.historical_data[ticker], consolidated_W.historical_data[ticker]) for t, ticker in enumerate(consolidated_W.historical_data.columns)])

    return res

beta = 0.5

res = calcul_somme(beta=beta)
res 
